In [2]:
import os
from dotenv import load_dotenv, get_key
load_dotenv()

True

In [9]:
prompt_keywords = r"""
You are a technical expert in generating keywords for searching Google Paptent Database for a given idea.
Return the list of comma seperated keywords for the following idea

**IDEA:** {topic}

**Note:**
1. Keywords should be comma seperated.
2. Do not add any comments except the keywords.
3. You can include most applicable synonymous technical terms for keywords.
"""

prompt_query = r"""
You are a technical expert in generating a effective query string for Google Patent Database Search 
for a given idea and also a set of keywords.
Return the list of comma seperated query string that can effectively return the list of best matched patents
from the patents.google.com patents database.

**IDEA:** {topic}

**Note:**
1. Query string should be comma seperated.
2. Do not add any comments except the query string.
3. At least three query string should be returned.
4. All keywords should be distributed in the complex queries.
5. You can use AND, OR, XOR, SAME, ADJ, NEAR, ), ( and more advance operator for generating query string.

**Output Format:**
((Wearable) AND (Noise Cancellation)),
((Wearable OR Mobile) AND (Noise (Cancellation OR Suppression))
"""

In [10]:
import pandas as pd
from duckduckgo_search import DDGS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate

llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=os.getenv('GEMINI_API_KEY'),
    temperature=0.1
)

def generate_keywords(topic):
    prompt = PromptTemplate.from_template(prompt_keywords)
    chain = prompt | llm
    result = chain.invoke(
        {
            "topic": topic
        }
    )
    print(result.content)
    keywords = result.content.split(',')
    keywords = [keyword.strip() for keyword in keywords]
    return keywords

def generate_queries(topic, keywords):
    prompt = PromptTemplate.from_template(prompt_query)
    chain = prompt | llm
    result = chain.invoke(
        {
            "topic": topic,
            "keywords": keywords,
        }
    )
    print(result.content)
    queries = result.content.split(',')
    queries = [q.strip() for q in queries]
    return queries

def search_duckduckgo(query):
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=10)
    return results

def parse_results_to_dataframe(results):
    data = []
    for result in results:
        data.append({
            'Title': result.get('title'),
            'URL': result.get('href'),
            'Description': result.get('body')
        })
    df = pd.DataFrame(data)
    return df

def search_prior_art(query):
    full_query = f"https://patents.google.com?q={query}"
    print(full_query)
    results = search_duckduckgo(full_query)
    df = parse_results_to_dataframe(results)
    return df

topic = "A new method for efficient solar energy conversion using nanotechnology."
#df = search_prior_art(description)
#print(df)

keywords = generate_keywords(topic)
queries = generate_queries(topic, keywords)
for query in queries:
    print(query)
    sr = search_prior_art(query)
    print(sr)
    break

Nanotechnology, Solar energy conversion, Photovoltaics, Solar cells, Quantum dots, Nanocrystals, Semiconductor nanoparticles, Light absorption, Charge separation, Charge transport, Efficiency, Renewable energy
((Nanotechnology OR Nano) AND (Solar Energy) AND (Conversion OR Efficiency)),
((Nanotechnology OR Nano) AND (Solar Energy) AND (Conversion OR Efficiency) AND (Patent)),
((Nanotechnology OR Nano) AND (Solar Energy) AND (Conversion OR Efficiency) AND (Patent) AND (Recent))
((Nanotechnology OR Nano) AND (Solar Energy) AND (Conversion OR Efficiency))
https://patents.google.com?q=((Nanotechnology OR Nano) AND (Solar Energy) AND (Conversion OR Efficiency))
                                               Title  \
0                                     Google Patents   
1  US20180261705A1 - Photodetector cell and solar...   
2  US9293255B2 - Solid state energy storage devic...   
3  Google Patents - Wikipedia, the free encyclopedia   
4  Search for patents | USPTO - United States Pat...   

In [11]:
sr

,Title,URL,Description
0,Google Patents,https://patents.google.com/,"Search within the title, abstract, claims, or ..."
1,US20180261705A1 - Photodetector cell and solar...,https://patents.google.com/patent/US2018026170...,Tsakalakos et al. [8] studied p-n junction bas...
2,US9293255B2 - Solid state energy storage devic...,https://patents.google.com/patent/US9293255B2/en,Energy storage devices and methods of producti...
3,"Google Patents - Wikipedia, the free encyclopedia",https://en.wikipedia.org/wiki/Google_Patents,Wikipedia entry for Google Patents.Google Pate...
4,Search for patents | USPTO - United States Pat...,https://www.uspto.gov/patents/search,Patent Public Search. The Patent Public Search...
5,Patent Public Search | USPTO,https://www.uspto.gov/patents/search/patent-pu...,The https:// ensures that you are connecting t...
6,Searching for patents | epo.org - European Pat...,https://www.epo.org/en/searching-for-patents,"Technical, legal and business information abou..."
7,US6362718B1 - Motionless electromagnetic gener...,https://patents.google.com/patent/US6362718B1/en,Because the electromagnetic generator 10 is se...
8,United States Patent and Trademark Office,https://www.uspto.gov/,The https: // ensures that you ... Patents. Pa...
9,Large-scale high-temperature solar energy stor...,https://www.sciencedirect.com/science/article/...,It is widely accepted that the massive deploym...
